## Homework

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

/bin/bash: gdrive: command not found


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv', parse_dates=['timestamp'])

In [4]:
# df = pd.read_csv('/content/drive/MyDrive/Tinkoff Generation ML/Datasets/train.csv', parse_dates=["timestamp"])

In [5]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [4]:
TARGET = 'price'
X = df.drop(TARGET, axis=1)
y = df[TARGET]

In [5]:
from sklearn.model_selection import train_test_split

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [6]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(X.columns).difference(set(cat_columns + drop_columns)))

In [7]:
X = X.drop(drop_columns, axis=1)

In [8]:
def get_true_mean(column, ceil=False):
  nans = X[column].isna().sum()
  mean = X[column].sum() / (X[column].size - nans)

  if ceil:
    return np.round(mean)
  return mean

In [9]:
for column in num_columns:
  na_value = get_true_mean(column, True)
  X[column] = X[column].fillna(na_value)

In [10]:
for column in cat_columns:
  X[column] = X[column].fillna(X[column].mode())

In [11]:
X = pd.get_dummies(X, columns=cat_columns)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

Попробуем дропнуть NaN из категоральных признаков

In [13]:
df_droped = df.copy()

for column in cat_columns:
  df_droped = df_droped[df_droped[column].notna()]

df.shape, df_droped.shape

((20000, 292), (11082, 292))

In [14]:
X_droped = df_droped.drop(TARGET, axis=1)
y_droped = df_droped[TARGET]
X_droped = X_droped.drop(drop_columns, axis=1)

In [15]:
for column in num_columns:
  na_value = get_true_mean(column, True)
  X_droped[column] = X_droped[column].fillna(na_value)

In [16]:
X_droped = pd.get_dummies(X_droped, columns=cat_columns)

In [17]:
X_droped_train, X_droped_test, y_droped_train, y_droped_test = train_test_split(X_droped, y_droped, train_size=0.8)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [18]:
from sklearn.tree import DecisionTreeRegressor


model = DecisionTreeRegressor().fit(X_train, y_train)
predictions = model.predict(X_test)

Проверьте качество на отложенной выборке.

In [19]:
from sklearn.metrics import mean_squared_error

print(f'RMSE Decision Tree: {mean_squared_error(y_test, predictions, squared=False):.2f}')

RMSE Decision Tree: 3633223.79


In [20]:
model_droped = DecisionTreeRegressor().fit(X_droped_train, y_droped_train)
predictions_droped = model_droped.predict(X_droped_test)

In [21]:
print(f'RMSE Decision Tree: {mean_squared_error(y_droped_test, predictions_droped, squared=False):.2f}')

RMSE Decision Tree: 4041167.51


Средняя ошибка в 3 ляма? Ничего, потерпим

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [20]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

C:\Users\Micha\AppData\Local\Temp\ipykernel_29144\3560215100.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitch_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [21]:
df["month"] = df.timestamp.dt.month
df["day_of_week"] = df.timestamp.dt.weekday

In [22]:
def get_floor_max_floor_ratio(elem):
  if np.isnan(elem['max_floor']) or elem['max_floor'] == 0:
    return 1

  ratio = elem['floor'] / elem['max_floor']
  return ratio

def get_kitchen_flat_ratio(elem):
  if np.isnan(elem['kitch_sq']) or elem['full_sq'] == 0:
    return 0.1

  ratio = elem['kitch_sq'] / elem['full_sq']
  return ratio

df['floor_max_floor_ratio'] = df.apply(get_floor_max_floor_ratio, axis=1)
df['kitchen_full_ratio'] = df.apply(get_kitchen_flat_ratio, axis=1)

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [23]:
X = df.drop(TARGET, axis=1)
X = X.drop(drop_columns, axis=1)
y = df[TARGET]

In [24]:
for column in num_columns:
  na_value = get_true_mean(column, True)
  X[column] = X[column].fillna(na_value)

In [25]:
for column in cat_columns:
  X[column] = X[column].fillna(X[column].mode())

In [26]:
X = pd.get_dummies(X, columns=cat_columns)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [29]:
!pip install catboost
!pip install xgboost
!pip install lightgbm

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [29]:
%%time
tree = DecisionTreeRegressor().fit(X_train, y_train)
forest = RandomForestRegressor().fit(X_train, y_train)
cboost = CatBoostRegressor(task_type='GPU', devices='0-2').fit(X_train, y_train)
xboost = XGBRegressor(tree_method='gpu_hist').fit(X_train, y_train)
lgbm = LGBMRegressor().fit(X_train, y_train)

Learning rate set to 0.058083
0:	learn: 4556774.2089338	total: 25.4ms	remaining: 25.4s
1:	learn: 4424343.3202239	total: 58.5ms	remaining: 29.2s
2:	learn: 4298066.5298017	total: 81.9ms	remaining: 27.2s
3:	learn: 4182072.9183668	total: 107ms	remaining: 26.6s
4:	learn: 4074335.2234236	total: 131ms	remaining: 26.1s
5:	learn: 3969997.5703870	total: 155ms	remaining: 25.7s
6:	learn: 3875834.8607881	total: 179ms	remaining: 25.3s
7:	learn: 3788225.9851445	total: 202ms	remaining: 25s
8:	learn: 3707629.9130601	total: 226ms	remaining: 24.9s
9:	learn: 3634775.2419066	total: 250ms	remaining: 24.7s
10:	learn: 3566684.4357496	total: 277ms	remaining: 24.9s
11:	learn: 3503245.5611736	total: 303ms	remaining: 24.9s
12:	learn: 3440662.1573382	total: 327ms	remaining: 24.9s
13:	learn: 3386638.1926923	total: 353ms	remaining: 24.9s
14:	learn: 3336955.3075079	total: 376ms	remaining: 24.7s
15:	learn: 3288907.8526805	total: 399ms	remaining: 24.6s
16:	learn: 3242201.2747811	total: 424ms	remaining: 24.5s
17:	learn:

In [30]:
tree_pred = tree.predict(X_test)
forest_pred = forest.predict(X_test)
cboost_pred = cboost.predict(X_test)
xboost_pred = xboost.predict(X_test)
lgbm_pred = lgbm.predict(X_test)

In [31]:
print(f'RMSE Дерева: {mean_squared_error(y_test, tree_pred, squared=False):.2f}')
print(f'RMSE Случайного леса: {mean_squared_error(y_test, forest_pred, squared=False):.2f}')
print(f'RMSE CatBoost: {mean_squared_error(y_test, cboost_pred, squared=False):.2f}')
print(f'RMSE XGBoost: {mean_squared_error(y_test, xboost_pred, squared=False):.2f}')
print(f'RMSE LightGBM: {mean_squared_error(y_test, lgbm_pred, squared=False):.2f}')

RMSE Дерева: 3600023.22
RMSE Случайного леса: 2638222.89
RMSE CatBoost: 2536035.86
RMSE XGBoost: 2680575.20
RMSE LightGBM: 2601661.48


CatBoost показал лучший результат(ВПЁРЕД РОССИЯ 🇷🇺🔥🥇) без подбора гиперпараметров, теперь попробуем подобрать их.

In [32]:
from sklearn.model_selection import GridSearchCV 

In [63]:
params_tree = {'max_depth': [None, 3, 5, 7, 12], 'min_samples_leaf': [1, 5, 10, 14, 17, 20, 25]}
params_forest = {'n_estimators': [30, 50], 'max_depth': [3, 5, 7, 12], 'max_features':['sqrt', 'log2'],
                 'min_samples_leaf': [5, 10, 14]}
params_cboost = {'depth': [3, 5, 12], 'learning_rate': [0.04, 0.06, 0.08], 'iterations': [400, 700, 1000]}
params_xboost = {'max_depth': [3, 6, 12], 'sampling_method': ['gradient_based'], 'subsample': [0.3, 0.5, 1], 'max_bin': [10]}

In [30]:
clf_tree = GridSearchCV(DecisionTreeRegressor(), params_tree, scoring='neg_root_mean_squared_error')
clf_tree.fit(X_train, y_train)

print(f'Лучшие параметры: {clf_tree.best_params_}')
prediction_tree = clf_tree.predict(X_test)
print(f'RMSE Дерева: {mean_squared_error(y_test, prediction_tree, squared=False):.2f}')

Лучшие параметры: {'max_depth': 7, 'min_samples_leaf': 20}
RMSE Дерева: 3065575.89


In [45]:
clf_forest = GridSearchCV(RandomForestRegressor(), params_forest, scoring='neg_root_mean_squared_error')
clf_forest.fit(X_train, y_train)

print(f'Лучшие параметры: {clf_forest.best_params_}')
prediction_forest = clf_forest.predict(X_test)
print(f'RMSE Дерева: {mean_squared_error(y_test, prediction_forest, squared=False):.2f}')

Лучшие параметры: {'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 50}
RMSE Дерева: 3027628.45


In [52]:
catboost = CatBoostRegressor(task_type='GPU', devices='0', max_bin=1).grid_search(params_cboost, X_train, y_train)

0:	learn: 8290547.8136480	test: 7992042.9864716	best: 7992042.9864716 (0)	total: 9.2ms	remaining: 3.67s
1:	learn: 8035230.7618228	test: 7738677.8063060	best: 7738677.8063060 (1)	total: 17.7ms	remaining: 3.53s
2:	learn: 7792379.5632485	test: 7497689.3222930	best: 7497689.3222930 (2)	total: 26.1ms	remaining: 3.46s
3:	learn: 7561769.6275759	test: 7267856.7122090	best: 7267856.7122090 (3)	total: 34.5ms	remaining: 3.42s
4:	learn: 7341766.3779488	test: 7048121.8237390	best: 7048121.8237390 (4)	total: 42.8ms	remaining: 3.38s
5:	learn: 7134914.2825245	test: 6841848.5447119	best: 6841848.5447119 (5)	total: 51.7ms	remaining: 3.39s
6:	learn: 6937107.8791301	test: 6643785.1316320	best: 6643785.1316320 (6)	total: 60.6ms	remaining: 3.4s
7:	learn: 6749528.3479202	test: 6455596.6435574	best: 6455596.6435574 (7)	total: 78.3ms	remaining: 3.84s
8:	learn: 6572291.4507590	test: 6279742.2687430	best: 6279742.2687430 (8)	total: 86.8ms	remaining: 3.77s
9:	learn: 6403053.9698451	test: 6110505.8777198	best: 611

In [53]:
print(catboost['params'])

{'depth': 12, 'iterations': 400, 'learning_rate': 0.08}


In [57]:
best_catboost = CatBoostRegressor(task_type='GPU', devices='0', depth=12, learning_rate=0.08, iterations=400, silent=True).fit(X_train, y_train)

In [58]:
prediction_catboost = best_catboost.predict(X_test)
print(f'RMSE Дерева: {mean_squared_error(y_test, prediction_catboost, squared=False):.2f}')

RMSE Дерева: 2622323.37


In [64]:
clf_xgb = GridSearchCV(XGBRegressor(tree_method='gpu_hist'), params_xboost, scoring='neg_root_mean_squared_error')
clf_xgb.fit(X_train, y_train)

print(f'Лучшие параметры: {clf_xgb.best_params_}')
prediction_xgb = clf_xgb.predict(X_test)
print(f'RMSE XGBoost: {mean_squared_error(y_test, prediction_xgb, squared=False):.2f}')

Лучшие параметры: {'max_bin': 10, 'max_depth': 3, 'sampling_method': 'gradient_based', 'subsample': 0.5}
RMSE XGBoost: 2693497.76


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [34]:
X_train.shape

(16000, 1070)

In [78]:
class Ensemble:
    def __init__(self):
        self.model_invest = CatBoostRegressor(task_type='GPU', devices='0', depth=12, learning_rate=0.06, iterations=400, silent=True)
        # self.model_owner = CatBoostRegressor(task_type='GPU', devices='0', depth=12, learning_rate=0.06, iterations=400, silent=True)
        # self.model_invest = XGBRegressor(tree_method='gpu_hist', max_depth=5, sampling_method='gradient_based', subsample=0.5)
        self.model_owner = XGBRegressor(tree_method='gpu_hist', max_depth=5, sampling_method='gradient_based', subsample=0.5)

    def fit(self, X, y):
        indices_invest = np.where(X['product_type_Investment'] == 1)
        self.model_invest = self.model_invest.fit(X.iloc[indices_invest], y.iloc[indices_invest])

        indices_own = np.where(X['product_type_Investment'] == 0)
        self.model_owner = self.model_owner.fit(X.iloc[indices_own], y.iloc[indices_own])

        return self

    def predict(self, X):
        answer = np.zeros(X.shape[0])

        indices_invest = np.where(X['product_type_Investment'] == 1)
        invest_predictions = self.model_invest.predict(X.iloc[indices_invest])

        indices_own = np.where(X['product_type_Investment'] == 0)
        owner_predictions = self.model_owner.predict(X.iloc[indices_own])

        for pos, price in zip(indices_invest, invest_predictions):
            answer[pos] = price

        for pos, price in zip(indices_own, owner_predictions):
            answer[pos] = price

        return answer

In [79]:
ensemble = Ensemble().fit(X_train, y_train)
predictions = ensemble.predict(X_test)

In [80]:
print('RMSE ансамбля:')
print(f'{mean_squared_error(y_test, predictions, squared=False):.2f}')

RMSE ансамбля:
4615791.52


In [58]:
model = XGBRegressor(tree_method='gpu_hist', max_depth=4, sampling_method='gradient_based', subsample=0.5).fit(X_train, y_train)
predictions = model.predict(X_test)

In [60]:
print('RMSE XGBoost:')
print(f'{mean_squared_error(y_test, predictions, squared=False):.2f}')

RMSE XGBoost:
2550603.03


Ансамбль показывает худший результат, причём гораздо и независимо от моделей

![](../../../Downloads/mem-pochemu.jpg)

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [63]:
from sklearn.linear_model import HuberRegressor

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [71]:
class CoolerEnsemble:
    def __init__(self, w1=0.5, w2=0.5):
        self.model_invest1 = CatBoostRegressor(task_type='GPU', devices='0', depth=12, learning_rate=0.06, iterations=400, silent=True)
        self.model_invest2 = HuberRegressor(max_iter=500)

        self.w1 = w1
        self.w2 = w2

        # self.model_owner = CatBoostRegressor(task_type='GPU', devices='0', depth=12, learning_rate=0.06, iterations=400, silent=True)
        # self.model_invest = XGBRegressor(tree_method='gpu_hist', max_depth=5, sampling_method='gradient_based', subsample=0.5)
        self.model_owner = XGBRegressor(tree_method='gpu_hist', max_depth=5, sampling_method='gradient_based', subsample=0.5)

    def fit(self, X, y):
        indices_invest = np.where(X['product_type_Investment'] == 1)
        self.model_invest1 = self.model_invest1.fit(X.iloc[indices_invest], y.iloc[indices_invest])
        self.model_invest2 = self.model_invest2.fit(X.iloc[indices_invest], y.iloc[indices_invest])

        indices_own = np.where(X['product_type_Investment'] == 0)
        self.model_owner = self.model_owner.fit(X.iloc[indices_own], y.iloc[indices_own])

        return self

    def predict(self, X):
        answer = np.zeros(X.shape[0])

        indices_invest = np.where(X['product_type_Investment'] == 1)
        invest_predictions1 = self.model_invest1.predict(X.iloc[indices_invest])
        invest_predictions2 = self.model_invest2.predict(X.iloc[indices_invest])

        indices_own = np.where(X['product_type_Investment'] == 0)
        owner_predictions = self.model_owner.predict(X.iloc[indices_own])

        for pos, price1, price2 in zip(indices_invest, invest_predictions1, invest_predictions2):
            answer[pos] = price1 * self.w1 + price2 * self.w2

        for pos, price in zip(indices_own, owner_predictions):
            answer[pos] = price

        return answer

In [77]:
weights = np.arange(0.1, 1, 0.1)

for w in weights:
    ensemble = CoolerEnsemble(w, 1-w).fit(X_train, y_train)
    predictions = ensemble.predict(X_test)

    print(f'RMSE крутого ансамбля при w1={w}, w2={1-w}:')
    print(f'{mean_squared_error(y_test, predictions, squared=False):.2f}')

RMSE крутого ансамбля при w1=0.1, w2=0.9:
4684392.47
RMSE крутого ансамбля при w1=0.2, w2=0.8:
4671036.19
RMSE крутого ансамбля при w1=0.30000000000000004, w2=0.7:
4659092.35
RMSE крутого ансамбля при w1=0.4, w2=0.6:
4648571.84
RMSE крутого ансамбля при w1=0.5, w2=0.5:
4639484.34
RMSE крутого ансамбля при w1=0.6, w2=0.4:
4631838.28
RMSE крутого ансамбля при w1=0.7000000000000001, w2=0.29999999999999993:
4625640.82
RMSE крутого ансамбля при w1=0.8, w2=0.19999999999999996:
4620897.78
RMSE крутого ансамбля при w1=0.9, w2=0.09999999999999998:
4617613.64
